## Spark Wrangling Exercises

In [1]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType

spark = SparkSession.builder.getOrCreate()

### Data Acquisition
- These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.
- This exercises uses the 'case.csv', 'dept.csv', and 'source.csv' files from the san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

In [2]:
# first I need to download all three files as csv, and store locally

In [3]:
# case
# Read in CSV file 
case = (spark.read.csv("case.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [4]:
case.show(1, False, True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [5]:
# dept
# Read in CSV file 
dept = (spark.read.csv("dept.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [6]:
dept.show(1, False, True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [7]:
# source
# Read in CSV file 
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [8]:
source.show(1, False, True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



2. Let's see how writing to the local disk works in spark:
- Write the code necessary to store the source data in both csv and json format, store these as 'sources_csv' and 'sources_json'
- Inspect your folder structure. What do you notice?

In [9]:
# let's write as a csv first and document what happens...
(
    source.write.format('csv')
    .mode('overwrite')
    .option('header', 'true')
    .save('sources_csv')
)

In [10]:
# now as a json...
(
    source.write.format('json')
    .mode('overwrite')
    .option('header', 'true')
    .save('sources_json')
)

In [11]:
# It appears both files are too large to be stored locally, 
# and that only a small portion was saved

3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

#### Case

In [12]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [13]:
# case
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [14]:
# Change 'SLA_due_date' to 'case_due_date'
# Change 'SLA_days' tp 'case_days_alotted'

case = case.withColumnRenamed('SLA_due_date', 'case_due_date')
case = case.withColumnRenamed('SLA_days', 'case_days')

In [15]:
# did it work as expected?
case.show(1, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 case_due_date        | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 case_days            | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [16]:
# 'case_late' and 'case_closed' to boolean
case = case.withColumn('case_closed', expr('case_closed == "YES"'))\
.withColumn('case_late', expr('case_late == "YES"'))

In [17]:
# did it work as expected?
case.show(1, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 case_due_date        | 9/26/20 0:42                         
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 case_days            | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [18]:
# council district as a string
case = case.withColumn('council_district', col('council_district').cast('string'))

In [19]:
# 'case_opened_date', 'case_closed_date', 'case_due_date' to datetime format

fmt = "M/d/yy H:mm"

case = case.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))\
.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))\
.withColumn('case_due_date', to_timestamp('case_due_date', fmt))

In [20]:
# did it work as expected?
case.show(1, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 case_days            | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [21]:
# trim and lowercase address info

case = case.withColumn('request_address', trim(lower(case.request_address)))

In [22]:
# use format_string function to pad zeros for council_district

case = case.withColumn('council_district', format_string('%03d', col('council_district').cast('int')))

In [23]:
# create a new column for zipcode:

case = case.withColumn('zipcode', regexp_extract('request_address', r"(\d+$)", 1))

In [24]:
case.show(1, False, True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 case_days            | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 zipcode              | 78207                                
only sho

#### Department

In [25]:
dept.show(1, vertical=True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [26]:
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [27]:
# convert 'dept_subject_to_SLA' to boolean
dept = dept.withColumn('dept_subject_to_SLA', expr('dept_subject_to_SLA == "YES"'))

In [28]:
# good to go?
dept.show(1, vertical=True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | true             
only showing top 1 row



#### Source

In [29]:
source.show(1, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [30]:
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [31]:
# good to go

### Data Wrangling

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [32]:
# filter to only look at currently open cases
# use a datediff, or sort by days_open with ascending=False

In [33]:
#create three new columns 'case_age', 'days_to_closed', 'case_lifetime'

case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

In [34]:
# filtering where case is still open
# sorting by age of case, in descending order
# not truncating any values
# showing vertically, the top 1 case
case.filter(case.case_closed == 'false').sort(desc('case_age')).show(1, False, True)

-RECORD 0------------------------------------------------------
 case_id              | 1013225646                             
 case_opened_date     | 2017-01-01 13:48:00                    
 case_closed_date     | null                                   
 case_due_date        | 2017-01-17 08:30:00                    
 case_late            | true                                   
 num_days_late        | 348.6458333                            
 case_closed          | false                                  
 dept_division        | Code Enforcement                       
 service_request_type | No Address Posted                      
 case_days            | 15.77859954                            
 case_status          | Open                                   
 source_id            | svcCRMSS                               
 request_address      | 7299  shadow ridge, san antonio, 78250 
 council_district     | 006                                    
 zipcode              | 78250           

2. How many Stray Animal cases are there?

In [35]:
case.select(case.service_request_type).show(5)

+--------------------+
|service_request_type|
+--------------------+
|        Stray Animal|
|Removal Of Obstru...|
|Removal Of Obstru...|
|Front Or Side Yar...|
|Animal Cruelty(Cr...|
+--------------------+
only showing top 5 rows



In [36]:
case.filter(case.service_request_type == 'Stray Animal').count()

26760

3. How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?

In [37]:
case.filter(case.dept_division == 'Field Operations').where(case['service_request_type'] != 'Officer Standby').count()

113902

4. Convert the `council_district` column to a string column.

In [38]:
# already done did it...
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('case_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('zipcode', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int')]

5. Extract the year from the `case_closed_date` column.

In [39]:
# create a new column for year:

case = case.withColumn('year', regexp_extract('case_closed_date', r"^(\d{4})", 1))

case.select('year').show(5)

+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 5 rows



6. Convert `num_days_late` from days to hours in new columns `num_hours_late`.

In [40]:
# create a new column for year:
case = case.withColumn('num_hours_late', col('num_days_late')*24)

case.select(round('num_hours_late', 2)).show(5)

+------------------------+
|round(num_hours_late, 2)|
+------------------------+
|               -23964.21|
|                   -48.3|
|                  -72.54|
|                 -360.28|
|                    8.93|
+------------------------+
only showing top 5 rows



7. Join the case data with the source and department data.

In [41]:
# join the df and dept dataframe using 'dept_division' as common key
# drop columns as needed (keep standardized_dept_name)
# convert dept_subject_to_SLA to boolean

df = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
    .join(source, 'source_id', 'left')
    
)

In [42]:
df.show(1, False, True)

-RECORD 0----------------------------------------------------
 source_id            | svcCRMLS                             
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 service_request_type | Stray Animal                         
 case_days            | 999.0                                
 case_status          | Closed                               
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 zipcode              | 78207                                
 case_age             | 1315                                 
 days_to

8. Are there any cases that do not have a request source?

In [43]:
df.filter(df.source_id.isNull()).count()

0

9. What are the top 10 service request types in terms of number of requests?

In [44]:
df.groupBy('service_request_type').count().show(10, False)

+--------------------------------------+-----+
|service_request_type                  |count|
+--------------------------------------+-----+
|Minimum Housing-Owner Occupied        |8617 |
|Tree Removal                          |298  |
|Service Information                   |164  |
|Sign Maintenance                      |82   |
|Park Building Maint Invest            |48   |
|Brush Property Damage                 |186  |
|Graffiti: Private Property (Corridors)|8525 |
|Traffic Sign Graffiti                 |2150 |
|License Renewal Invoice               |1394 |
|Used/Scrap Tire Facility Registration |19   |
+--------------------------------------+-----+
only showing top 10 rows



10. What are the top 10 service request types in terms of average days late?

In [45]:
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



11. Does number of days late depend on department?


In [46]:
(
    df.where('case_late')
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



12. How do number of days late depend on department and request type?

In [48]:
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)

)

AnalysisException: cannot resolve '`dept_name`' given input columns: [case_age, case_closed, case_closed_date, case_days, case_due_date, case_id, case_late, case_lifetime, case_opened_date, case_status, council_district, days_to_closed, department, dept_subject_to_SLA, num_days_late, num_hours_late, request_address, service_request_type, source_id, source_username, year, zipcode];
'Aggregate ['dept_name], ['dept_name, avg(num_days_late#21) AS days_late#1517, count(num_days_late#21) AS n_cases_late#1519L]
+- Filter case_late#386: boolean
   +- Project [source_id#27, case_id#16, case_opened_date#487, case_closed_date#502, case_due_date#517, case_late#386, num_days_late#21, case_closed#371, service_request_type#24, case_days#285, case_status#26, request_address#603, council_district#618, zipcode#633, case_age#782, days_to_closed#799, case_lifetime#817, year#979, num_hours_late#1006, department#1124, dept_subject_to_SLA#1146, source_username#177]
      +- Join LeftOuter, (source_id#27 = so